In [ ]:
# !pip install git+https://github.com/SethPoulsen/sklearn-crfsuite.git@add-missing-attributes#egg=sklearn_crfsuite

In [ ]:
# !pip install --force-reinstall --no-dependencies "scikit-learn==0.24.2"

In [7]:
# import pandas as pd
import json
import sys
import copy


from itertools import chain

import sklearn
from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

### Load data

In [8]:
list_of_stc = []
list_of_list_token_of_stc = []
list_of_list_label_of_stc = []
list_of_list_POStag = []

list_of_sentiment = []
list_of_stc_id = []

with open("./file12345_isPOStag_NER_phase_filter_UNK_oversampling.txt", "r", encoding = "utf-8") as f:
    while f:
        info = f.readline()[:-1] # bỏ dấu xuống dòng
        stc = f.readline()[:-1]
        token = f.readline()[:-1]
        label = f.readline()[:-1]
        postag = f.readline()[:-1]

        # check hết file
        if info == "" and stc == "" and label == "":
            break

        # add idx, add sentiment
        info_split = info.split("\t")
        list_of_stc_id.append(int(info_split[0]))
        sentiment = json.loads(info_split[1])

        # add sentiment
        if sentiment[0] == 0:
            list_of_sentiment.append([1])
        elif sentiment[0] in [1, 2]:
            list_of_sentiment.append([2])
        else:
            assert sentiment[0] in [-1, -2]
            list_of_sentiment.append([0])

        # add stc, add label, add postag
        list_of_stc.append(json.loads(stc))
        list_of_list_token_of_stc.append(json.loads(token))
        list_of_list_label_of_stc.append(json.loads(label))
        list_of_list_POStag.append(json.loads(postag))

In [9]:
# Check how many labels are there in the dataset
unique_labels = set()

for list_lb in list_of_list_label_of_stc:
  [unique_labels.add(lb) for lb in list_lb if lb not in unique_labels]

print(unique_labels)

{'B-obj', 'O', 'B-ap', 'I-sub', 'I-obj', 'B-sub', 'I-ap'}


In [10]:
import random

def shuffle_data(data:list):
    random.shuffle(data)


def build_data_CEE_CRF(list_of_token_stc, list_of_POS_tag, list_CEE_label, is_unify_entity = False, is_shuffle_data = False):
    data_set = []
    CEE_label_dict = {
        "B-sub": "SUB",
        "I-sub": "SUB",
        "B-obj": "OBJ",
        "I-obj": "OBJ",
        "B-ap": "AP",
        "I-ap": "AP",
        "O": "O"
    }

    for index, CEE_label in enumerate(list_CEE_label):
        pos_tag = list_of_POS_tag[index]
        tokens = list_of_token_stc[index]
        
        assert len(pos_tag) == len(tokens)
        assert len(tokens) ==  len(CEE_label)

        list_for_stc = []
        for i, dict_ in enumerate(pos_tag):
            if is_unify_entity:
                list_for_stc.append(tuple([tokens[i], pos_tag[i], CEE_label_dict[CEE_label[i]]]))
            else:
                assert CEE_label[i] in CEE_label_dict.keys()
                list_for_stc.append(tuple([tokens[i], pos_tag[i], CEE_label_dict[i]]))
        data_set.append(list_for_stc)

    if is_shuffle_data:
        shuffle_data(data_set)
    end_train_idx = 9000
    return data_set[:end_train_idx], data_set[end_train_idx:]

data_NER_CRF_train, data_NER_CRF_test  = build_data_CEE_CRF(list_of_list_token_of_stc, list_of_list_POStag, list_of_list_label_of_stc, True)

In [11]:
data_NER_CRF_train[0]

[('Nhìn', 'V', 'O'),
 ('vào', 'E', 'O'),
 ('những', 'L', 'O'),
 ('thông_số', 'N', 'AP'),
 ('kích_thước', 'N', 'AP'),
 ('trên', 'N', 'O'),
 (',', 'CH', 'O'),
 ('VinFast_Lux_SA', 'Np', 'SUB'),
 ('2.0', 'M', 'SUB'),
 ('đang', 'R', 'O'),
 ('nhỉnh', 'V', 'O'),
 ('hơn', 'A', 'O'),
 ('Kia_Sorento', 'Np', 'OBJ'),
 ('về', 'E', 'O'),
 ('mọi', 'L', 'O'),
 ('chỉ_số', 'N', 'O'),
 (',', 'CH', 'O'),
 ('điều', 'N', 'O'),
 ('này', 'P', 'O'),
 ('giúp', 'V', 'O'),
 ('xe', 'N', 'O'),
 ('vạm_vỡ', 'V', 'O'),
 ('hơn', 'R', 'O'),
 ('khi', 'N', 'O'),
 ('nhìn', 'V', 'O'),
 ('từ', 'E', 'O'),
 ('phần', 'N', 'O'),
 ('thân', 'N', 'O')]

### Sentence Feature Representation

In [12]:
def _word2features_pos(sent, i, window):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:3]': postag[:3],
    }

    for w in range(1, window + 1):
        if i > 0:
            word = sent[i - w][0]
            postag = sent[i - w][1]
            features.update({
                '-{}:word.lower()'.format(w): word.lower(),
                '-{}:word.istitle()'.format(w): word.istitle(),
                '-{}:word.isupper()'.format(w): word.isupper(),
                '-{}:postag'.format(w): postag,
                '-{}:postag[:3]'.format(w): postag[:3],
            })
        else:
            features['-{}:BOS'.format(w)] = True

        if i < len(sent) - w:
            word1 = sent[i + w][0]
            postag1 = sent[i + w][1]
            features.update({
                '+{}:word.lower()'.format(w): word1.lower(),
                '+{}:word.istitle()'.format(w): word1.istitle(),
                '+{}:word.isupper()'.format(w): word1.isupper(),
                '+{}:postag'.format(w): postag1,
                '+{}:postag[:3]'.format(w): postag1[:3],
            })
        else:
            features['-{}:EOS'.format(w)] = True

    return features


def _word2features(sent, i, window):
    word = sent[i][0]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
    }
    for w in range(1, window + 1):
        if i > 0:
            word = sent[i - w][0]
            features.update({
                '-{}:word.lower()'.format(w): word.lower(),
                '-{}:word.istitle()'.format(w): word.istitle(),
                '-{}:word.isupper()'.format(w): word.isupper(),
            })
        else:
            features['-{}:BOS'.format(w)] = True

        if i < len(sent) - w:
            word1 = sent[i + w][0]
            features.update({
                '+{}:word.lower()'.format(w): word1.lower(),
                '+{}:word.istitle()'.format(w): word1.istitle(),
                '+{}:word.isupper()'.format(w): word1.isupper(),
            })
        else:
            features['-{}:EOS'.format(w)] = True
    return features


def word2features(sent, i, is_pos, window):
    if is_pos:
        return _word2features_pos(sent, i, window)
    return _word2features(sent, i, window)

def sent2features(sent, is_pos, window):
    return [word2features(sent, i, is_pos, window) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent, is_pos):
    if is_pos:
        return [token for token, postag, label in sent]
    return [token for token, label in sent]

### Evaluation Metrics

In [13]:
# Convert output to a single element presentation
def get_entity_output_of_CRF(y_test, y_pred, type_element, is_unify_entity = True):
    assert len(y_test) == len(y_pred), 'Độ dài dữ liệu test và pred không bằng nhau'

    test_list, pred_list = [], []
    for i, stc in enumerate(y_test):
        assert len(y_test[i]) ==  len(y_pred[i]), 'Độ dài câu văn test và pred không bằng nhau'
        
        test_stc, pred_stc = [], []
        for k, _ in enumerate(stc):
            test_stc.append(type_element) if y_test[i][k] == type_element else test_stc.append("O")
            pred_stc.append(type_element) if y_pred[i][k] == type_element else pred_stc.append("O")
            
        test_list.append(test_stc)
        pred_list.append(pred_stc)

    return test_list, pred_list

In [38]:
# Trả về 0 hoặc 1 nếu là exact hoặc binary
# Trả về độ dài nếu là proportional

def get_first_index(stc : list, element_type):
    for index, e in enumerate(stc):
        if e == element_type:
            return index
    return -1


def get_last_index(stc : list, element_type):
    for index, e in enumerate(stc):
        if e == element_type and index < len(stc) -1:
            if stc[index+1] in ['O', 0]:
                return index
        elif e == element_type and index == len(stc) -1:
            return index
    return -1


def get_span_index(stc : list, element_type):
    return [get_first_index(stc, element_type), get_last_index(stc, element_type)]


def get_overlapping(span_test, span_pred):
    # span_test = [-1, -1]            # test
    # span_pred = [-1, -1]          # pred
    set1 = set(range(span_test[0], span_test[1] + 1))
    set2 = set(range(span_pred[0], span_pred[1] + 1))
    overlap = set1.intersection(set2)

    len_span_test = len(set1) if -1 not in set1 else 0  # không có span thì len_span_test là 0 chứ không phải là 1
    len_span_pred = len(set2) if -1 not in set2 else 0
    len_overlap = len(overlap) if ((-1 not in set1) and (-1 not in set2)) else 0 # Giải thích: cả 2 thằng đều là [-1, -1](không tồn tại) thì bản chất chúng không hề overlap

    # print(f'Kích thước test: {len_span_test}')
    # print(f'Kích thước pred: {len_span_pred}')
    # print(f'Kích thước overlap: {len_overlap}')

    return [len_span_test, len_span_pred, len_overlap]


def evaluate_full_named_entity(y_test, y_pred, element_type, evaluate_type, ovelapping_ratio = 0.75):
    assert evaluate_type in ["exact", "partial", "binary"], "Wrong type of evaluate"
    assert element_type in [1, "SUB", "AP", "OBJ"], "Wrong type of element"
    assert len(y_test) == len(y_pred)

    total_1_test, total_1_pred_true, total_1_pred = 0, 0, 0  #toàn bộ tập test

    for i in range(len(y_test)):
        assert len(y_test[i]) ==  len(y_pred[i]), 'Kích thước câu văn test và pred không bằng nhau'
        span_test = get_span_index(y_test[i], element_type)
        span_pred = get_span_index(y_pred[i], element_type)
        len_span_test, len_span_pred, len_overlap = get_overlapping(span_test, span_pred)

        if len_span_test > 0:   # x[0] != -1 and x[1] != -1 Tức là nhãn có element này trong câu văn này
            total_1_test += 1

        if len_span_pred > 0:   # y[0] !=-1 and y[1] != -1. Tức là đoán có element
            total_1_pred +=1

        if len_span_pred > 0:   # y[0] !=-1 and y[1] != -1
            if evaluate_type == 'binary':
                if len_overlap > 0:
                    total_1_pred_true += 1
            
            elif evaluate_type == 'exact':
                if len_span_test == len_overlap and len_span_pred == len_overlap:
                    total_1_pred_true += 1

            elif evaluate_type == 'partial' and ovelapping_ratio is not None:
                if len_span_test != 0 and (len_overlap >= ovelapping_ratio * len_span_test):
                    total_1_pred_true += 1
            else:
                raise Exception("No evaluate type found. Or ovelapping_ratio is None")

    # print(total_1_test, total_1_pred, total_1_pred_true)
    precision = total_1_pred_true / total_1_pred
    recall = total_1_pred_true / total_1_test
    f1_score = 2*precision*recall/(precision+recall)
    # print(f'Precision: {round(precision, 4)*100}, Recall: {round(recall,4)*100}, F1-Score: {round(f1_score,4)*100}')
    print(f'Precision: {round(precision, 4)}, Recall: {round(recall,4)}, F1-Score: {round(f1_score,4)}')

In [26]:
# if evaluate_type == 'binary':
#     if len_span_pred == 0 and len_span_test == 0:
#         return 1
#     if len_overlap > 0:
#         return 1
#     return 0

# elif evaluate_type == 'exact':
#     if len_span_pred == 0 and len_span_test == 0:
#         return 1
#     elif len_span_test == len_overlap and len_span_pred == len_overlap:
#         return 1
#     return 0

# elif evaluate_type == 'partial' and ovelapping_ratio is not None:
#     if len_span_pred == 0 and len_span_test == 0:                   
#         return 1
#     elif len_overlap / len_span_test >= ovelapping_ratio:
#         return 1
# else:
#     raise Exception("No evaluate type found") 

### Main

In [34]:
def feature_present(is_pos : bool, window_size):
    X_train = [sent2features(sent = s, is_pos = is_pos, window = window_size) for s in data_NER_CRF_train]
    y_train = [sent2labels(sent = s) for s in data_NER_CRF_train]
    X_test = [sent2features(sent = s, is_pos = is_pos, window = window_size) for s in data_NER_CRF_test]
    y_test = [sent2labels(sent = s) for s in data_NER_CRF_test]
    return X_train, X_test, y_train, y_test


def train(X_train, y_train):
    crf_model = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=100,
        all_possible_transitions=False,
    )
    crf_model.fit(X_train, y_train)
    return crf_model


def test(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    for element_type in ["SUB", "OBJ", "AP"]:
        print(f'===== {element_type} =====')
        for evaluate_type in ["binary", "partial", "exact"]:
            print(f'{evaluate_type.upper()}', end = ':  ')
            test, pred = get_entity_output_of_CRF(y_test, y_pred, element_type)
            evaluate_full_named_entity(test, pred, element_type, evaluate_type)

In [41]:
for window_size in range(1,4):
    print(f'\n====================== Window_size : {window_size} ======================')
    X_train, X_test, y_train, y_test = feature_present(is_pos=True, window_size=window_size)
    crf_model = train(X_train, y_train)
    test(crf_model, X_test, y_test)
    del crf_model
    
    # print(metrics.flat_classification_report(y_test, pred, labels=['SUB', 'OBJ', 'AP'], digits=4))

    # ====================================================================================
    print('\nNo POS-tag:')
    X_train, X_test, y_train, y_test = feature_present(is_pos=False, window_size=window_size)
    crf_model = train(X_train, y_train)
    test(crf_model, X_test, y_test)
    del crf_model

    # print(metrics.flat_classification_report(y_test, pred, labels=['SUB', 'OBJ', 'AP'], digits=4))


====================== Window_size : 1 ======================


===== SUB =====
BINARY:  Precision: 0.9226, Recall: 0.7561, F1-Score: 0.8311
PARTIAL:  Precision: 0.9107, Recall: 0.7463, F1-Score: 0.8204
EXACT:  Precision: 0.869, Recall: 0.7122, F1-Score: 0.7828
===== OBJ =====
BINARY:  Precision: 0.8468, Recall: 0.6731, F1-Score: 0.75
PARTIAL:  Precision: 0.8306, Recall: 0.6603, F1-Score: 0.7357
EXACT:  Precision: 0.75, Recall: 0.5962, F1-Score: 0.6643
===== AP =====
BINARY:  Precision: 0.8333, Recall: 0.5851, F1-Score: 0.6875
PARTIAL:  Precision: 0.7727, Recall: 0.5426, F1-Score: 0.6375
EXACT:  Precision: 0.7197, Recall: 0.5053, F1-Score: 0.5938

No POS-tag:
===== SUB =====
BINARY:  Precision: 0.9222, Recall: 0.7512, F1-Score: 0.828
PARTIAL:  Precision: 0.8982, Recall: 0.7317, F1-Score: 0.8065
EXACT:  Precision: 0.8443, Recall: 0.6878, F1-Score: 0.7581
===== OBJ =====
BINARY:  Precision: 0.8425, Recall: 0.6859, F1-Score: 0.7562
PARTIAL:  Precision: 0.8189, Recall: 0.6667, F1-Score: 0.735
EXACT:  Precision: 0.7402, Recall: 0.6026, F1-Score: 0.6643


In [23]:
# print(evaluate_full_named_entity(sub_test, sub_pred, 'SUB', 'binary'))
# print(evaluate_full_named_entity(ap_test, ap_pred, 'AP', 'binary'))
# print(evaluate_full_named_entity(obj_test, obj_pred, 'OBJ', 'binary'))


(86.9, 71.22, 78.28)
(71.97, 50.529999999999994, 59.38)
(75.0, 59.62, 66.43)


In [17]:
sub_test, sub_pred = get_entity_output_of_CRF(y_test, y_pred, "SUB")
print(sub_test)

[['O', 'O', 'O', 'O', 'O', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'SUB', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'SUB', 'SUB', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'SUB', 'SUB', 'SUB', 'SUB', 'SUB', 'O', 'O', 'O'], ['O', 'SUB', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'SUB', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'SUB', 'SUB', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',